<a href="https://colab.research.google.com/github/pagarsky/nn2022/blob/master/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторна робота №6
#### з дисципліни "Нейронні мережі"
#### студента групи КН-31/2
#### Пагарського Олексія
#### Варіант №7

In [ ]:
N = 7
i1, i2, i3 = N - 1, N + 29, N + 59

In [ ]:
import random
import os
import pathlib
from distutils.dir_util import copy_tree

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

!pip install tensorflow_hub
import tensorflow_hub as hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Виконати вирішення задачі класифікації для 3 класів з набору даних food101 з використанням різних моделей нейронних мереж:


In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

!tar xvf food-101.tar.gz

In [ ]:
!ls -la food-101/images

classes = []
for dirpath, dirnames, filenames in os.walk("food-101/images/"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
    classes.append(dirpath)

In [ ]:
dirs = [
    classes[i1],
    classes[i2],
    classes[i3]
]
classes = [dir.split('/')[-1] for dir in dirs]
print(classes)

['beef_carpaccio', 'escargots', 'cup_cakes']


In [ ]:
for cls in classes:
    # print(cls)
    for img in os.walk(f'food-101/images/{cls}/'):
        copy_tree(f'food-101/images/{cls}', f'./images/{cls}/')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    './images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_data = train_datagen.flow_from_directory(
    './images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


### CNN модель

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(20, 3, activation='selu', input_shape=(224, 224, 3)),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Conv2D(15, 5, activation='selu'),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Conv2D(10, 7, activation='selu'),
  tf.keras.layers.MaxPool2D(),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(3, activation='sigmoid')
])

model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 20)      560       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 20)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 107, 107, 15)      7515      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 53, 15)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 47, 47, 10)        7360      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 10)       0

In [ ]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, verbose=True),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

history = model.fit(
    train_data,
    epochs=10,
    batch_size=64,
    steps_per_epoch=len(train_data),
    validation_data=val_data,
    validation_steps=len(val_data),
    callbacks=callbacks,
)

print('\n'*5)
loss, accuracy = model.evaluate(val_data)
print(f'loss: {loss}\naccuracy: {accuracy}')

Epoch 1/10
75/75 [==============================] - 51s 527ms/step - loss: 1.0347 - accuracy: 0.4921 - val_loss: 0.9423 - val_accuracy: 0.5633 - lr: 0.0010
Epoch 2/10
75/75 [==============================] - 40s 529ms/step - loss: 0.9695 - accuracy: 0.5521 - val_loss: 0.9429 - val_accuracy: 0.5883 - lr: 0.0010
Epoch 3/10
75/75 [==============================] - 39s 521ms/step - loss: 0.9127 - accuracy: 0.5896 - val_loss: 0.9204 - val_accuracy: 0.5783 - lr: 0.0010
Epoch 4/10
75/75 [==============================] - 39s 520ms/step - loss: 0.9184 - accuracy: 0.5888 - val_loss: 0.9214 - val_accuracy: 0.5633 - lr: 0.0010
Epoch 5/10
75/75 [==============================] - 39s 524ms/step - loss: 0.9058 - accuracy: 0.6008 - val_loss: 0.9097 - val_accuracy: 0.5817 - lr: 0.0010
Epoch 6/10
75/75 [==============================] - 40s 530ms/step - loss: 0.9053 - accuracy: 0.6033 - val_loss: 0.8882 - val_accuracy: 0.6217 - lr: 0.0010
Epoch 7/10
75/75 [==============================] - 39s 522ms/st

### Resnet модель

In [ ]:
resnet = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/5",
                   trainable=False),

    tf.keras.layers.Dense(10, activation="selu"),
    tf.keras.layers.Dense(3, activation="sigmoid"),
])
resnet.build([None, 224, 224, 3])  # Batch input shape.

resnet.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

resnet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              42605504  
                                                                 
 dense_1 (Dense)             (None, 10)                20490     
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 42,626,027
Trainable params: 20,523
Non-trainable params: 42,605,504
_________________________________________________________________


In [ ]:
history = resnet.fit(
    train_data,
    epochs=10,
    batch_size=64,
    steps_per_epoch=len(train_data),
    validation_data=val_data,
    validation_steps=len(val_data),
    callbacks=callbacks,
)

print('\n'*5)
loss, accuracy = resnet.evaluate(val_data)
print(f'loss: {loss}\naccuracy: {accuracy}')

Epoch 1/10
75/75 [==============================] - 53s 602ms/step - loss: 0.3516 - accuracy: 0.8746 - val_loss: 0.1808 - val_accuracy: 0.9433 - lr: 0.0010
Epoch 2/10
75/75 [==============================] - 43s 574ms/step - loss: 0.1665 - accuracy: 0.9429 - val_loss: 0.1515 - val_accuracy: 0.9367 - lr: 0.0010
Epoch 3/10
75/75 [==============================] - 43s 578ms/step - loss: 0.1394 - accuracy: 0.9492 - val_loss: 0.1361 - val_accuracy: 0.9450 - lr: 0.0010
Epoch 4/10
75/75 [==============================] - 43s 572ms/step - loss: 0.1103 - accuracy: 0.9613 - val_loss: 0.1365 - val_accuracy: 0.9533 - lr: 0.0010
Epoch 5/10
75/75 [==============================] - 43s 572ms/step - loss: 0.0954 - accuracy: 0.9663 - val_loss: 0.1451 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 6/10
75/75 [==============================] - ETA: 0s - loss: 0.0896 - accuracy: 0.9671
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
75/75 [==============================] - 43s 570m

### Efficientnet модель

In [15]:
efficientnet = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                   trainable=False),
                   
    tf.keras.layers.Dense(10, activation="selu"),
    tf.keras.layers.Dense(3, activation="sigmoid"),
])
efficientnet.build([None, 224, 224, 3])  # Batch input shape.

efficientnet.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

efficientnet.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              4049564   
                                                                 
 dense_5 (Dense)             (None, 10)                12810     
                                                                 
 dense_6 (Dense)             (None, 3)                 33        
                                                                 
Total params: 4,062,407
Trainable params: 12,843
Non-trainable params: 4,049,564
_________________________________________________________________


In [16]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, verbose=True),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

history = efficientnet.fit(
    train_data,
    epochs=10,
    batch_size=64,
    steps_per_epoch=len(train_data),
    validation_data=val_data,
    validation_steps=len(val_data),
    callbacks=callbacks,
)

print('\n'*5)
loss, accuracy = efficientnet.evaluate(val_data)
print(f'loss: {loss}\naccuracy: {accuracy}')

Epoch 1/10
75/75 [==============================] - 51s 583ms/step - loss: 0.2856 - accuracy: 0.9067 - val_loss: 0.1253 - val_accuracy: 0.9650 - lr: 0.0010
Epoch 2/10
75/75 [==============================] - 43s 570ms/step - loss: 0.1023 - accuracy: 0.9683 - val_loss: 0.1055 - val_accuracy: 0.9600 - lr: 0.0010
Epoch 3/10
75/75 [==============================] - 43s 575ms/step - loss: 0.0769 - accuracy: 0.9792 - val_loss: 0.0964 - val_accuracy: 0.9717 - lr: 0.0010
Epoch 4/10
75/75 [==============================] - 41s 550ms/step - loss: 0.0581 - accuracy: 0.9829 - val_loss: 0.0998 - val_accuracy: 0.9650 - lr: 0.0010
Epoch 5/10
75/75 [==============================] - 41s 552ms/step - loss: 0.0536 - accuracy: 0.9817 - val_loss: 0.0982 - val_accuracy: 0.9683 - lr: 0.0010
Epoch 6/10
75/75 [==============================] - 41s 550ms/step - loss: 0.0427 - accuracy: 0.9846 - val_loss: 0.0937 - val_accuracy: 0.9683 - lr: 0.0010
Epoch 7/10
75/75 [==============================] - 41s 552ms/st

### Висновок

При виконанні Лабораторної роботи 6, ми підготували тренувальний та валідаційний датасети для класифікації їжі за обраними варіантами.

Застосували згорткові нейронні мережі для класифікації зображень з ЛР5. Також обрали претреновані моделі ResNet і EffecientNet, які були підготовані на датасеті ImageNet. В верхніх шарах цих моделей було відключено тренування і треновані лише останні два повнозв'язних шари.

- CNN ~65%
- ResNet ~95%
- EfficientNet ~96.7%